In [15]:
%%pyspark
df = spark.read.load('abfss://fille@lakez.dfs.core.windows.net/salesproject/sales.csv', format='csv'
## If header exists uncomment line below
,header=True
)
display(df.limit(10))

StatementMeta(sparkes, 0, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8633a533-7e16-4960-bc80-c2bb7719e183)

In [2]:
!pip install mlflow azureml-sdk scikit-learn


StatementMeta(sparkes, 0, 3, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 75.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 120.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.6/565.6 kB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 134.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 117.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 16.9 MB/s eta 0:00:00
     ━

In [3]:
import mlflow
mlflow.set_experiment("Synapse-MLflow-Tracking")


StatementMeta(sparkes, 0, 4, Finished, Available, Finished)

2024/10/14 06:47:00 INFO mlflow.tracking.fluent: Experiment with name 'Synapse-MLflow-Tracking' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///mnt/var/hadoop/tmp/nm-local-dir/usercache/trusted-service-user/appcache/application_1728888002783_0001/container_1728888002783_0001_01_000001/mlruns/139907574266693331', creation_time=1728888420167, experiment_id='139907574266693331', last_update_time=1728888420167, lifecycle_stage='active', name='Synapse-MLflow-Tracking', tags={}>

In [26]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
import mlflow.spark

# Assuming 'df' is your original DataFrame and DEALSIZE is your target variable
# Identify categorical features and your target variable
target_col = 'DEALSIZE'
categorical_cols = [col for col in df.columns if col != target_col]  # Exclude target column

# Step 1: Index the target column DEALSIZE
indexer_target = StringIndexer(inputCol=target_col, outputCol=target_col + "_indexed")

# Step 2: Index categorical features
indexers = [StringIndexer(inputCol=col, outputCol=col + "_indexed") for col in categorical_cols]

# Step 3: OneHotEncode the indexed categorical columns
encoders = [OneHotEncoder(inputCol=col + "_indexed", outputCol=col + "_encoded") for col in categorical_cols]

# Step 4: Create a feature vector using VectorAssembler
feature_columns = [col + "_encoded" for col in categorical_cols]  # Use encoded columns
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Step 5: Create a pipeline that includes all transformations
pipeline = Pipeline(stages=[indexer_target] + indexers + encoders + [assembler])

# Step 6: Fit the pipeline to the DataFrame and transform it
df_transformed = pipeline.fit(df).transform(df)

# Step 7: Split data into training and testing sets
train_data, test_data = df_transformed.randomSplit([0.8, 0.2], seed=1234)

# Step 8: Train a RandomForest model
rf = RandomForestClassifier(labelCol=target_col + "_indexed", featuresCol="features", maxBins=3000)
model = rf.fit(train_data)

# Step 9: Make predictions
predictions = model.transform(test_data)

# Step 10: Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol=target_col + "_indexed", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

# Step 11: Log parameters and metrics with MLflow
with mlflow.start_run():
    mlflow.log_param("n_estimators", 100)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.spark.log_model(model, "random_forest_model")

print("Model training and logging complete.")


StatementMeta(sparkes, 0, 27, Finished, Available, Finished)

2024/10/14 07:32:00 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp7r23e2_8/model, flavor: spark). Fall back to return ['pyspark==3.4.1.5.3.20230713']. Set logging level to DEBUG to see the full traceback. 
2024/10/14 07:32:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model training and logging complete.


In [10]:
import logging
logging.getLogger("mlflow").setLevel(logging.DEBUG)


StatementMeta(sparkes, 9, 4, Finished, Available, Finished)

In [9]:
model_uri = "runs:/bc8bc2be311046f7b1a1c81ead692da4/random_forest_model"
mlflow.register_model(model_uri, "Synapse-RF-Model")


StatementMeta(sparkes, 9, 3, Finished, Available, Finished)

Successfully registered model 'Synapse-RF-Model'.
Created version '1' of model 'Synapse-RF-Model'.


<ModelVersion: aliases=[], creation_timestamp=1729097238828, current_stage='None', description=None, last_updated_timestamp=1729097238828, name='Synapse-RF-Model', run_id='bc8bc2be311046f7b1a1c81ead692da4', run_link=None, source='file:///mnt/var/hadoop/tmp/nm-local-dir/usercache/trusted-service-user/appcache/application_1729097055673_0001/container_1729097055673_0001_01_000001/mlruns/0/bc8bc2be311046f7b1a1c81ead692da4/artifacts/random_forest_model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [8]:
import mlflow

# Start an MLflow run
with mlflow.start_run() as run:
    # Get the run ID from the active run
    run_id = run.info.run_id
    print("Run ID:", run_id)

# You can now use `run_id` for logging metrics, params, or querying runs


StatementMeta(sparkes, 9, 2, Finished, Available, Finished)

Run ID: bc8bc2be311046f7b1a1c81ead692da4


In [19]:
from azureml.core import Workspace

# Provide the full path to the config.json file
ws = Workspace.from_config(path='abfss://fille@lakez.dfs.core.windows.net/Project/config.json')



print("Workspace name:", ws.name)


StatementMeta(sparkes, 9, 13, Finished, Available, Finished)

UserErrorException: UserErrorException:
	Message: The workspace configuration file config.json, could not be found in /mnt/var/hadoop/tmp/nm-local-dir/usercache/trusted-service-user/appcache/application_1729097055673_0001/container_1729097055673_0001_01_000001/abfss:/fille@lakez.dfs.core.windows.net/Project/config.json or its parent directories. Please check whether the workspace configuration file exists, or provide the full path to the configuration file as an argument. You can download a configuration file for your workspace, via http://ml.azure.com and clicking on the name of your workspace in the right top.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "The workspace configuration file config.json, could not be found in /mnt/var/hadoop/tmp/nm-local-dir/usercache/trusted-service-user/appcache/application_1729097055673_0001/container_1729097055673_0001_01_000001/abfss:/fille@lakez.dfs.core.windows.net/Project/config.json or its

In [11]:
from azureml.core import Workspace, Model

ws = Workspace.from_config()
model = Model.register(model_path="model.pkl",
                       model_name="my_model",
                       workspace=ws)


StatementMeta(sparkes, 9, 5, Finished, Available, Finished)

UserErrorException: UserErrorException:
	Message: The workspace configuration file config.json, could not be found in /mnt/var/hadoop/tmp/nm-local-dir/usercache/trusted-service-user/appcache/application_1729097055673_0001/container_1729097055673_0001_01_000001 or its parent directories. Please check whether the workspace configuration file exists, or provide the full path to the configuration file as an argument. You can download a configuration file for your workspace, via http://ml.azure.com and clicking on the name of your workspace in the right top.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "The workspace configuration file config.json, could not be found in /mnt/var/hadoop/tmp/nm-local-dir/usercache/trusted-service-user/appcache/application_1729097055673_0001/container_1729097055673_0001_01_000001 or its parent directories. Please check whether the workspace configuration file exists, or provide the full path to the confi